In [1]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
J_tr, P_tr = data_collection(robot=panda, N=config.N_train)
J_ts, P_ts = data_collection(robot=panda, N=config.N_test)
F = posture_feature_extraction(J_tr)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()
knn = get_knn(P_tr=P_tr)

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne.pickle
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth


In [3]:
# mu = torch.randn(size=(7,))
mu = torch.zeros(size=(7,))

In [ ]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0.01)
test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=iflow, knn=knn, K=NUM_SAMPLES, print_report=True)

In [ ]:
nflow = get_nflow_model(flow=flow)
test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=nflow, knn=knn, K=NUM_SAMPLES, print_report=True)

In [5]:
total_params = sum(
	param.numel() for param in flow.parameters()
)
print(total_params)

15964263


In [4]:
Path_dir = sample_P_path(robot=panda, load_time='0816002845')

/home/luca/ikpflow/data/panda/trajectory/0816002845/ load successfully.


In [45]:
def path_following(
    robot,
    Path_dir: str,
    model,
    knn,
    F,
    num_traj: int = 3,
) -> None:
    """
    path following generation for our method
    
    Parameters
    ----------
    robot : _type_
        robotic arm
    Path_dir : str
        path to ee Path, generated by sample_P_path
    model : _type_
        flow, iflow, or nflow
    knn : _type_
        knn of P_train
    F : _type_
        F_train
    num_traj : int, optional
        the number of generated joint trajectory samples, by default 3
    """

    def load_and_plot(exp_traj_path: str, ee_path: np.array):
        if os.path.exists(path=exp_traj_path):
            robot.plot(qs=qs)
        else:
            print(f"{exp_traj_path} does not exist !")

    Path = load_numpy(file_path=Path_dir + "ee_traj.npy")
    Path = Path[:, :3]

    ref_F = nearest_neighbor_F(knn, np.atleast_2d(Path[0]), F, n_neighbors=40_0000) # knn
    # ref_F = F
    # ref_F = rand_F(Path[0], F)
    
    exp_path = lambda idx: Path_dir + f"exp_{idx}.npy"
    
    rand_idxs = np.random.randint(low=0, high=len(ref_F), size=num_traj)
    
    # rand_idxs = list(range(num_traj))
    for i, rand in enumerate(rand_idxs):
        # ref_F = rand_F(Path[0], F)
        
        df, qs = sample_J_traj(Path, ref_F[rand], model, robot) 
        print("="*6 + str(rand) + f"=({ref_F[rand]})" + "="*6)
        print(df.describe())
        save_numpy(file_path=exp_path(i), arr=qs)

    # for i in range(num_traj):
    #     load_and_plot(exp_traj_path=exp_path(i), ee_path=Path)

In [46]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0)
nflow = get_nflow_model(flow=flow, shrink_ratio=0.2)

In [47]:
path_following(robot=panda, Path_dir=Path_dir, model=iflow, knn=knn, F=F, num_traj=3)

======237507=([ 0.34613949 -0.49850781 -0.97209309 -0.77293011])======
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.004503      32.037345
std           0.005157      63.856751
min           0.000699       0.281547
25%           0.002206       5.649660
50%           0.003106      18.931518
75%           0.004377      29.771475
max           0.024761     296.944977
======70381=([-0.6071551   0.49470081 -0.86732943  1.08439235])======
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.002559       9.185927
std           0.001563       5.861401
min           0.000581       0.266429
25%           0.001076       4.653818
50%           0.002381       9.553985
75%           0.003832      13.829163
max           0.005061      21.070053
======316412=([-0.08868511 -0.63099161 -0.97494804 -1.02603948])======
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.003050      

In [11]:
path_following(robot=panda, Path_dir=Path_dir, model=nflow, knn=knn, F=F)

       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.001615      48.994754
std           0.000586      19.788968
min           0.000328      21.897610
25%           0.001247      39.527138
50%           0.001596      42.718073
75%           0.002032      53.504387
max           0.002788      92.368759
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.001724      33.983042
std           0.000923      12.528462
min           0.000521      12.911550
25%           0.001027      24.164518
50%           0.001665      34.077032
75%           0.002172      45.908035
max           0.003786      55.757313
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.001445      45.234315
std           0.000660      21.635774
min           0.000423      17.352730
25%           0.000955      30.166533
50%           0.001461      43.818924
75%           0.001965      57.768088
max         

In [ ]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

In [ ]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

In [ ]:
target_poses = load_numpy(file_path=traj_dir + 'ee_traj.npy')
# traj_dir, target_poses = get_target_poses(robot=panda)
exp_dir = traj_dir + f'exp_5_single_obj/'
data = {fn: [] for fn in file_names}

# -> unrefined solutions
mean_position_errors = np.zeros((num_ikflow_trails,))
mean_ang_errs = np.zeros((num_ikflow_trails,))
for ikflow_i in range(num_ikflow_trails):
    solutions, position_errors, _, _, _, _ = ik_solver.solve_n_poses(
        target_poses, 
        refine_solutions=False, 
        return_detailed=True)

    iksols = solutions.detach().cpu().numpy()
    df = eval_J_traj(robot=panda, J_traj=solutions.to('cpu'), P_path=target_poses, position_errors=position_errors)
    mean_position_errors[ikflow_i], mean_ang_errs[ikflow_i]  = df.mean().values
data['ikflow_l2'] = mean_position_errors
data['ikflow_ang'] = mean_ang_errs    


In [ ]:
df = pd.DataFrame(mean_position_errors)
df.describe()

In [ ]:
df = pd.DataFrame(mean_ang_errs)
df.describe()